In [1]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("andrewmvd/trip-advisor-hotel-reviews")

print("Path to dataset files:", path)

100%|██████████| 5.14M/5.14M [00:00<00:00, 86.9MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/andrewmvd/trip-advisor-hotel-reviews/versions/2


In [2]:
import pandas as pd
import os
path = kagglehub.dataset_download("andrewmvd/trip-advisor-hotel-reviews")

files = os.listdir(path)
print("Файлы в датасете:", files)


Файлы в датасете: ['tripadvisor_hotel_reviews.csv']


In [3]:
dataset_file = os.path.join(path, "tripadvisor_hotel_reviews.csv")
df = pd.read_csv(dataset_file)
df

,Review,Rating
0,nice hotel expensive parking got good deal sta...,4
1,ok nothing special charge diamond member hilto...,2
2,nice rooms not 4* experience hotel monaco seat...,3
3,"unique, great stay, wonderful time hotel monac...",5
4,"great stay great stay, went seahawk game aweso...",5
...,...,...
20486,"best kept secret 3rd time staying charm, not 5...",5
20487,great location price view hotel great quick pl...,4
20488,"ok just looks nice modern outside, desk staff ...",2
20489,hotel theft ruined vacation hotel opened sept ...,1


In [4]:
import torch
from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np
import torch.nn as nn
import torch.optim as optim

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20491 entries, 0 to 20490
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Review  20491 non-null  object
 1   Rating  20491 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 320.3+ KB


In [6]:
X = df['Review'].values
y = df['Rating'].values
y = y - 1

In [7]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(max_features=1000)
X = vectorizer.fit_transform(X).toarray()

In [8]:
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.4, stratify=y, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, stratify=y_temp, random_state=42)

In [9]:
class CustomDataset(Dataset):
    def __init__(self, features, labels):
        self.features = features
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return torch.tensor(self.features[idx], dtype=torch.float32), torch.tensor(self.labels[idx], dtype=torch.long)

In [10]:
train_dataset = CustomDataset(X_train, y_train)
val_dataset = CustomDataset(X_val, y_val)
test_dataset = CustomDataset(X_test, y_test)

batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)


In [15]:
from tqdm import tqdm

In [47]:

num_classes = 5  # Поскольку у вас 5 классов
assert all(0 <= label < num_classes for label in y), "Некорректные метки!"

class ConvNet(nn.Module):
    def __init__(self, num_classes):
        super(ConvNet, self).__init__()
        self.conv1 = nn.Conv1d(in_channels=1, out_channels=16, kernel_size=3, padding=1)
        self.conv2 = nn.Conv1d(in_channels=16, out_channels=32, kernel_size=3, padding=1)
        self.pool = nn.MaxPool1d(kernel_size=2)
        self.fc1 = nn.Linear(32 * (1000 // 2 // 2), 128)
        self.fc2 = nn.Linear(128, num_classes)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.5)

    def forward(self, x):
        x = self.pool(self.relu(self.conv1(x)))
        x = self.pool(self.relu(self.conv2(x)))
        x = x.view(x.size(0), -1)
        x = self.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return x

num_epochs = 10
model = ConvNet(num_classes=num_classes)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

for epoch in range(num_epochs):
    print(f"Начинаем эпоху {epoch + 1}/{num_epochs}...")

    model.train()
    running_loss = 0.0
    for i, (inputs, labels) in enumerate(train_loader):
        inputs = inputs.unsqueeze(1)  # Добавляем измерение канала

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

        if i % 10 == 0:
            print(f"Батч {i}, Лосс: {loss.item():.4f}")

    print(f"Эпоха {epoch+1}/{num_epochs}, Лосс: {running_loss/len(train_loader):.4f}")

    # Валидация
    model.eval()
    val_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for i, (inputs, labels) in enumerate(val_loader):
            inputs = inputs.unsqueeze(1)  # Добавляем измерение канала
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            val_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    print(f"Валидация, Лосс: {val_loss/len(val_loader):.4f}, Точность: {100 * correct/total:.2f}%")



Начинаем эпоху 1/10...
Батч 0, Лосс: 1.6007
Батч 10, Лосс: 1.5706
Батч 20, Лосс: 1.2244
Батч 30, Лосс: 1.1739
Батч 40, Лосс: 1.2245
Батч 50, Лосс: 1.2371
Батч 60, Лосс: 1.3859
Батч 70, Лосс: 1.2054
Батч 80, Лосс: 1.0118
Батч 90, Лосс: 1.3820
Батч 100, Лосс: 1.2149
Батч 110, Лосс: 0.9203
Батч 120, Лосс: 1.1973
Батч 130, Лосс: 1.2068
Батч 140, Лосс: 1.1803
Батч 150, Лосс: 1.3179
Батч 160, Лосс: 0.9029
Батч 170, Лосс: 1.3324
Батч 180, Лосс: 1.2331
Батч 190, Лосс: 1.0968
Батч 200, Лосс: 1.0345
Батч 210, Лосс: 0.8863
Батч 220, Лосс: 1.0237
Батч 230, Лосс: 0.9028
Батч 240, Лосс: 1.1745
Батч 250, Лосс: 0.9371
Батч 260, Лосс: 0.8284
Батч 270, Лосс: 1.0298
Батч 280, Лосс: 1.1482
Батч 290, Лосс: 0.9841
Батч 300, Лосс: 0.9959
Батч 310, Лосс: 1.1540
Батч 320, Лосс: 0.8199
Батч 330, Лосс: 0.7869
Батч 340, Лосс: 0.9096
Батч 350, Лосс: 1.1377
Батч 360, Лосс: 1.3298
Батч 370, Лосс: 1.0278
Батч 380, Лосс: 0.8556
Эпоха 1/10, Лосс: 1.1041
Валидация, Лосс: 0.9668, Точность: 57.64%
Начинаем эпоху 2/10...
Б

In [48]:
def evaluate_model(model, test_loader):
    model.eval()
    correct = 0
    total = 0

    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs = inputs.unsqueeze(1)
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    print(f"Точность модели на тестовых данных: {accuracy:.2f}%")
    return accuracy

test_accuracy = evaluate_model(model, test_loader)


Точность модели на тестовых данных: 59.94%


In [53]:
class LSTMClassifier(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes, dropout=0.5):
        super(LSTMClassifier, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True, dropout=dropout)
        self.fc = nn.Linear(hidden_size, num_classes)
        self.dropout = nn.Dropout(dropout)
        self.relu = nn.ReLU()

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        out, _ = self.lstm(x, (h0, c0))

        out = out[:, -1, :]
        out = self.dropout(self.relu(out))
        out = self.fc(out)
        return out

input_size = 1000  # Длина вектора (количество признаков)
hidden_size = 128  # Размер скрытого состояния
num_layers = 2  # Количество LSTM-слоев
num_classes = len(np.unique(y))  # Количество классов (целевых меток)

model = LSTMClassifier(input_size, hidden_size, num_layers, num_classes).to('cpu')  # Используйте 'cuda', если доступен GPU
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

def train_lstm_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=10):
    for epoch in range(num_epochs):
        print(f"Начинается эпоха {epoch + 1}/{num_epochs}...")
        model.train()
        running_loss = 0.0
        for i, (inputs, labels) in enumerate(train_loader):
            inputs = inputs.unsqueeze(1)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()

            if i % 10 == 0:  # Лог каждые 10 батчей
                print(f"Порция {i}, Лосс: {loss.item():.4f}")

        print(f"Эпоха {epoch + 1}/{num_epochs}, Лосс: {running_loss / len(train_loader):.4f}")


        model.eval()
        val_loss = 0.0
        correct = 0
        total = 0
        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs = inputs.unsqueeze(1)
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                val_loss += loss.item()
                _, predicted = torch.max(outputs, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

        print(f"Валидационный Лосс: {val_loss / len(val_loader):.4f}, Точность: {100 * correct / total:.2f}%")

train_lstm_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=10)

def evaluate_model(model, test_loader):
    model.eval()
    correct = 0
    total = 0

    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs = inputs.unsqueeze(1)
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    print(f"Точность модели на тестовых данных: {accuracy:.2f}%")
    return accuracy

test_accuracy = evaluate_model(model, test_loader)


Начинается эпоха 1/10...
Порция 0, Лосс: 1.6244
Порция 10, Лосс: 1.5870
Порция 20, Лосс: 1.5405
Порция 30, Лосс: 1.3795
Порция 40, Лосс: 1.2358
Порция 50, Лосс: 1.4572
Порция 60, Лосс: 1.1349
Порция 70, Лосс: 1.2614
Порция 80, Лосс: 1.1672
Порция 90, Лосс: 1.2824
Порция 100, Лосс: 1.2178
Порция 110, Лосс: 1.3656
Порция 120, Лосс: 1.1535
Порция 130, Лосс: 1.0524
Порция 140, Лосс: 0.9328
Порция 150, Лосс: 1.0646
Порция 160, Лосс: 0.8616
Порция 170, Лосс: 1.1054
Порция 180, Лосс: 0.9260
Порция 190, Лосс: 0.8081
Порция 200, Лосс: 1.0668
Порция 210, Лосс: 1.0559
Порция 220, Лосс: 1.0774
Порция 230, Лосс: 1.1307
Порция 240, Лосс: 1.1017
Порция 250, Лосс: 0.9213
Порция 260, Лосс: 0.8311
Порция 270, Лосс: 0.8895
Порция 280, Лосс: 0.7881
Порция 290, Лосс: 0.8196
Порция 300, Лосс: 1.1326
Порция 310, Лосс: 0.7693
Порция 320, Лосс: 0.9816
Порция 330, Лосс: 1.1269
Порция 340, Лосс: 0.6112
Порция 350, Лосс: 0.7737
Порция 360, Лосс: 0.9100
Порция 370, Лосс: 0.8946
Порция 380, Лосс: 0.9208
Эпоха 1/10,

У модели LSTM точность выше при одинаковых эпохах